# SPY ETF Financial Data Mining and Feature Engineering

## Overview
This notebook implements the data extraction and feature engineering pipeline for the SPY ETF algorithmic trading system. It processes raw market data and creates technical indicators used for machine learning model training.

### Key Components:
- **Data Processing**: Fetches and cleans SPY market data from various sources
- **Technical Indicators**: Calculates ATR (Average True Range) and SMA (Standardized Moving Average)
- **Profit/Loss Labeling**: Creates target variables for supervised learning
- **Cross-Validation**: Implements time-series aware data splitting

### Trading Strategy Context:
This system focuses on long-only strategies with multiple timeframes (1-3 min, 3-5 min, 5-15 min) to capture different market dynamics.


## The FinData Class

**Purpose**: Central data container and processor for financial market data

### Class Structure:
- **data**: List of processed DataFrames containing market data with indicators
- **Parameters**: Configurable trading and indicator parameters
  - `makeATR_k`: Lookback period for ATR calculation (default: 15)
  - `p_lambda`: ATR multiplier for profit/loss calculation (default: 10)
  - `p_timeout`: Maximum holding period in minutes (default: 180)
  - `p_chi`: Profit-to-loss ratio multiplier (default: 1.5)
  - `SMA_ks`: List of SMA periods [7, 20, 50]

### Methods:
- `setParameters()`: Configure trading parameters
- `addDataFrame()`: Process raw data and add technical indicators
- `ovn()`: Overnight processing for different timeframes

In [6]:
class FinData:
    data = []
    makeATR_k = 15
    p_lambda = 10
    p_timeout =  15*12
    p_chi =1.5
    SMA_ks = [7,20,50]
    def __init__(self):
        pass
    def setParameters(self, makeATR_k_in: int, p_lambda_in: int, p_timeout_in: int,
                 p_chi_in: int, SMA_ks_in: list):
        self.makeATR_k = makeATR_k_in
        self.p_lambda = p_lambda_in
        self.p_timeout = p_timeout_in
        self.p_chi = p_chi_in
        self.SMA_ks = SMA_ks_in
        
    def addDataFrame(self,df):
        # I KNOW this isn't pretty, I swear I'll find a better way. This is a work in progress
        df = makeATR(df, self.makeATR_k)
        df = PL(df.dropna(),self.p_lambda,self.p_chi,self.p_timeout)
        for k in self.SMA_ks:
            df = SMA(df,k)
        self.data.append((df.dropna()))
    def ovn(self,df):
        for k in self.SMA_ks:
            df = SMA(df,k)
        self.data.append(df)


# Average True Range (ATR) Calculation

## Purpose
ATR measures market volatility and is crucial for dynamic position sizing and risk management. Unlike traditional ATR, this implementation uses price standard deviation for more responsive volatility measurement.

## Mathematical Foundation

### Traditional OHLC4 Calculation:
For any time period i: $OHLC_i = \frac{Open_i + High_i + Low_i + Close_i}{4}$

### Mean Price Calculation:
$$\mu_T = \frac{\sum_{i=T-k}^{T}(OHLC_i)}{k}$$

### Modified ATR Formula:
$$ATR_T = \sqrt{\frac{\sum_{i=T-k}^{T}(OHLC_i - \mu_T)^2}{k}}$$

## Implementation Notes
- **Lookback Period (k)**: Default 15 periods, optimized for 1-minute SPY data
- **Volatility Measure**: Uses standard deviation instead of true range for smoother signals
- **Risk Management**: ATR values directly feed into profit/loss target calculations
- **Adaptive Sizing**: Higher ATR = wider stops, lower ATR = tighter stops

In [7]:
import statistics as st
import pandas as pd

def makeATR(df,k):
    means = []
    df["ATR"] = df["Open"]
    
    for i,row in df.iterrows():
        # In order to prevent errors with variance calculations, this is incorporated to check if
        # the list has less than 2 elts
        df.loc[i,"ATR"] = pd.NA if len(means) <2 else st.stdev(means)
        means.append(row["Open"])
        if len(means) >k:
            means.pop(0) 
    return df

# Profit and Loss Target System

## Overview
This system creates binary classification targets (1 = profit, 0 = loss) by simulating forward-looking trades with dynamic profit/loss targets based on market volatility.

## Target Calculation Logic
For each timestamp, the system determines whether a hypothetical trade would hit profit or loss targets first, creating supervised learning labels.

## Key Parameters

### 1. ATR Multiplier (λ - Lambda)
Controls the sensitivity to market volatility:
- **Purpose**: Scales profit/loss targets based on current market conditions
- **Example**: If SPY ATR = 0.50 and λ = 10:
  - Base volatility adjustment = 10 × 0.50 = $5.00

### 2. Profit-to-Loss Ratio (χ - Chi)
Asymmetric risk-reward ratio:
- **Default**: 1.5 (profit targets 50% wider than loss targets)
- **Profit Target**: $Price_T + λ × χ × ATR$
- **Loss Target**: $Price_T - λ × ATR$

### 3. Timeout Period (t)
Maximum holding period before forced exit:
- **Purpose**: Prevents indefinite position holding
- **Logic**: If neither target hit within t periods, compare exit price to entry
- **Classification**: Profit (1) if $Price_{T+t} > Price_T$, Loss (0) otherwise

## Risk Management Benefits
- **Adaptive Targets**: Wider targets in volatile markets, tighter in calm markets
- **Asymmetric Rewards**: Larger profit targets improve risk-adjusted returns
- **Time Decay Protection**: Timeout prevents capital tie-up in stagnant positions

## Output Features Created

### 1. Binary Classification Target (PL)
- **Values**: 0 (loss) or 1 (profit)
- **Purpose**: Target variable for classification models (Random Forest, SVM, GLM)
- **Usage**: Enables probability-based trading decisions

### 2. Continuous Profit/Loss Value (pl_value)
- **Purpose**: Actual dollar P&L for regression models and backtesting
- **Scale**: Based on 1000-unit position size for standardization
- **Usage**: Direct performance measurement and regression target

### 3. Statistical Validation
- **Win Rate Tracking**: Monitors percentage of profitable trades
- **Window Size**: 1-hour rolling window (12 candles for 5-min data)
- **Model Calibration**: Ensures predicted probabilities align with actual outcomes
- **Reference**: See `strat_book.rmd` for detailed probability analysis

### 4. Quality Assurance
- **Posterior Probability Validation**: Compares model predictions to realized outcomes
- **Temporal Consistency**: Ensures stable win rates across different market regimes
- **Overfitting Detection**: Monitors for unrealistic prediction accuracy

In [4]:
from datetime import timedelta
def PL(df, l, chi, t):
    df["PL"] = pd.NA
    df["pl_value"] = pd.NA
    i = 0
    
    for index, row in df.iterrows():
        profit = row["Open"] + l*chi*row["ATR"]
        loss = row["Open"] - l*row["ATR"]
        
        t_copy = t
        idx = index
        j = i
        i+=1
        while t_copy >0 and j<len(df):
            if idx not in df.index:
                break
            if df.loc[idx, "Low"] <= loss:
                df.loc[index, "PL"] = 0
                df.loc[index,"pl_value"] = (loss - row["Open"])*1000
                break
            elif df.loc[idx, "High"] >= profit:
                df.loc[index, "PL"] = 1
                df.loc[index,"pl_value"] = (profit - row["Open"])*1000
                break
            j+=1
            idx+=timedelta(minutes=1)
            t_copy-=1
        if idx not in df.index:
            #print(index,idx)
            continue
        if t_copy == 0 and j<len(df):
            profit_loss_indicator = 1 if df.loc[idx, "Close"] > row["Open"] else 0
            df.loc[index, "PL"] = profit_loss_indicator
            df.loc[index,"pl_value"] = (df.loc[idx, "Close"] - row["Open"])*1000        
    return df

# Technical Indicators System

## Philosophy
Technical indicators provide quantitative measures of price momentum, trend strength, and market sentiment. This system uses normalized indicators that adapt to changing market conditions.

## Standardized Moving Average (SMA)

### Innovation: Price-Relative Normalization
Unlike traditional moving averages, this SMA is normalized by current price, making it scale-invariant and comparable across different price levels.

### Mathematical Definition
$$SMA_k = \frac{Open_T}{\frac{1}{k}\sum_{i=T-k}^{T-1}Close_i}$$

Where:
- $T$ = current time period
- $k$ = lookback period
- Values > 1.0 indicate price above historical average (bullish)
- Values < 1.0 indicate price below historical average (bearish)

### Multi-Timeframe Analysis
The system calculates SMA for three periods:
- **SMA_7**: Short-term momentum (1 week of 5-min bars)
- **SMA_20**: Medium-term trend (1 month of daily closes)
- **SMA_50**: Long-term trend (quarterly trend)

### Signal Interpretation
- **Trend Confirmation**: Multiple SMAs above 1.0 = strong uptrend
- **Momentum Divergence**: SMA_7 > SMA_20 > SMA_50 = accelerating uptrend
- **Mean Reversion**: Extreme SMA values (>1.05 or <0.95) suggest potential reversal

The following code will create SMA values for the data-set

In [5]:
def SMA(df,k):
    means = []
    colName = "SMA_k"+str(k)
    df[colName] = [0]*len(df)
    for i,row in df.iterrows():
        SMA_val = 1 if len(means) <1 else (row["Open"]/st.mean(means))
        df.loc[i,colName] = pd.NA if len(means) < 1 else SMA_val
        means.append(row["Open"])
        if len(means) > k:
            means.pop(0)
    return df

# Data Acquisition and Processing

## Data Sources
- **Primary**: Local SPY data files (1-minute and 15-minute intervals)
- **Backup**: Yahoo Finance API for additional data validation
- **Coverage**: Comprehensive historical data for robust backtesting

## Data Processing Pipeline

### 1. Raw Data Parsing
- Converts timestamp format from Unix milliseconds to datetime
- Handles OHLCV (Open, High, Low, Close, Volume) data structure
- Cleans and validates data integrity

### 2. Multiple Timeframe Generation
Creates datasets for different trading strategies:
- **1-3 minute**: Ultra-short scalping strategies
- **3-5 minute**: Short-term momentum capture
- **5-15 minute**: Medium-term trend following
- **8-60 minute**: Longer-term position strategies

### 3. Cross-Validation Framework
- **Time-Series Aware**: Maintains temporal order to prevent look-ahead bias
- **Walk-Forward Analysis**: Tests strategies on out-of-sample data
- **Multiple Folds**: Validates strategy robustness across different market conditions

### 4. Output Generation
Saves processed datasets as CSV files for R model training:

In [ ]:

import yfinance as yf
import pandas as pd
from datetime import datetime as dt
df = pd.DataFrame(columns=["Datetime","Open","Close","High","Low","Volume"])
with open ("../Trade infra/SPY_1m_data.txt", "r") as f:
   data = f.read()
   data = data.replace("{","")
   data = data.replace("}","")
   data = data.replace("[","")
   data = data.replace("]","")
   data = data.replace("]","")
   data = data.replace(" ","")
   data = data.replace("'","")
   data_list = data.split(",")
   i = -1
   for j,x in enumerate(data_list):
       if j%6 == 0:
           i+=1
       temp = x.split(":")
       if temp[0] == "datetime":
           temp= dt.fromtimestamp(int(temp[1])/1000).strftime('%Y-%m-%d %H:%M:%S')
           df.loc[i,temp[0].capitalize()]= dt.strptime(temp,'%Y-%m-%d %H:%M:%S')
       else: 
           df.loc[i,temp[0].capitalize()] = float(temp[1])
   df=df.set_index("Datetime")
df=df.set_index("2")
dfx = df.copy()
myData = FinData()
myData.addDataFrame(df.copy())
myData.data[0].to_csv("./train2.csv")

In [ ]:
import pandas as pd
import datetime as dt
# This is really unprofessional however when I developed it I didn't expect for it to be public
mydatax1=FinData()
mydatax2=FinData()
mydatax3=FinData()
mydatax4=FinData()
mydatax1.setParameters(15,3,5,1.5,[7,20,50])
mydatax2.setParameters(15,5,15,1.5,[7,20,50])
mydatax3.setParameters(15,8,60,1.5,[7,20,50])
mydatax4.setParameters(15,10,180,1.5,[7,20,50])
mydatax1.addDataFrame(dfx.copy())
mydatax2.addDataFrame(dfx.copy())
mydatax3.addDataFrame(dfx.copy())
mydatax4.addDataFrame(dfx.copy())
mydatax1.data[0].to_csv("./trainX3_5.csv")
mydatax2.data[0].to_csv("./trainX5_15.csv")
mydatax3.data[0].to_csv("./trainX8_60.csv")
mydatax4.data[0].to_csv("./trainX10_180.csv")




In [1]:
import sys
sys.modules[__name__].__dict__.clear()

In [ ]:
for ix in [[1,3]]:
    import pandas as pd
    from datetime import datetime as dt
    df = pd.DataFrame(columns=["Datetime","Open","Close","High","Low","Volume"])
    with open ("../Trade infra/train_txt_data/SPY_1m_data.txt", "r") as f:
        data = f.read()
        data = data.replace("{","")
        data = data.replace("}","")
        data = data.replace("[","")
        data = data.replace("]","")
        data = data.replace("]","")
        data = data.replace(" ","")
        data = data.replace("'","")
        data_list = data.split(",")
        i = -1
        for j,x in enumerate(data_list):
            if j%6 == 0:
                i+=1
            temp = x.split(":")
            if temp[0] == "datetime":
                temp= dt.fromtimestamp(int(temp[1])/1000).strftime('%Y-%m-%d %H:%M:%S')
                df.loc[i,temp[0].capitalize()]= dt.strptime(temp,'%Y-%m-%d %H:%M:%S')
            else: 
                df.loc[i,temp[0].capitalize()] = float(temp[1])
        df=df.set_index("Datetime")
    df=df.set_index("2")
    #dfx = df.copy()
    myData = FinData()
    if len(ix) == 2:
        myData.setParameters(15,ix[0],ix[1],1.5,[7,20,50])
        myData.addDataFrame(df.copy())
        myData.data[0].to_csv(f"./train{ix[0]}_{ix[1]}.csv")
    else:
        myData.addDataFrame(df.copy())
        myData.data[0].to_csv(f"./train2.csv")
    
    break

# 15 mins SPY ops

In [44]:

import yfinance as yf
import pandas as pd
from datetime import datetime as dt
df = pd.DataFrame(columns=["Datetime","Open","Close","High","Low","Volume"])
with open ("../Trade infra/SPY_15m_data.txt", "r") as f:
    data = f.read()
    data = data.replace("{","")
    data = data.replace("}","")
    data = data.replace("[","")
    data = data.replace("]","")
    data = data.replace("]","")
    data = data.replace(" ","")
    data = data.replace("'","")
    data_list = data.split(",")
    i = -1
    for j,x in enumerate(data_list):
        if j%6 == 0:
            i+=1
        temp = x.split(":")
        if temp[0] == "datetime":
            temp= dt.utcfromtimestamp(int(temp[1])/1000).strftime('%Y-%m-%d %H:%M:%S')
            df.loc[i,temp[0].capitalize()]= dt.strptime(temp,'%Y-%m-%d %H:%M:%S')
        else: 
            df.loc[i,temp[0].capitalize()] = float(temp[1])
    df=df.set_index("Datetime")
df=df.set_index("2")
myData = FinData()
myData.ovn(df)

myData.data[0].to_csv("train3.csv")
myData.data[0]

,Open,Close,High,Low,Volume,SMA_k7,SMA_k20,SMA_k50
2,,,,,,,,
2022-10-30 19:00:00,388.97,388.76,389.07,388.71,4818.0,NaN,NaN,NaN
2022-10-30 19:15:00,388.75,388.66,388.91,388.41,2226.0,0.999434,0.999434,0.999434
2022-10-30 19:30:00,388.71,388.38,388.71,388.28,2925.0,0.999614,0.999614,0.999614
2022-10-30 19:45:00,388.3,388.01,388.3,387.99,6485.0,0.998688,0.998688,0.998688
2022-10-30 20:00:00,388.03,388.11,388.14,388.03,930.0,0.998321,0.998321,0.998321
...,...,...,...,...,...,...,...,...
2023-07-14 17:45:00,449.25,449.3,449.39,449.25,11676.0,1.000757,1.000118,0.998820
2023-07-14 18:00:00,449.27,449.18,449.27,449.18,1549.0,1.000675,1.000227,0.998876
2023-07-14 18:15:00,449.16,449.16,449.16,449.16,500.0,1.000321,1.000045,0.998639


In [ ]:
import pandas as pd
d = {}
import datetime
df2 = pd.DataFrame()
#df2["diff"] = pd.NA

#sell all at open
nrow = 0
start = 0
for i,row in df.iterrows():
    #df.loc[nrow,""]
    if (13<i.hour or i.hour>=20) or (i.hour == 13 and i.minute < 30):
        continue
    elif i.hour == 19 and i.minute == 55:
        nrow+=1
        start = row["Open"]
    elif i.hour == 13 and i.minute == 30:
        continue
        df2.loc[nrow,"diff"] =row["Open"]- start
    else:
        for xr in df2.columns:
            df2.loc[nrow,str(xr)+"_"+str(i)] = row[xr]
df2
        
        
        

        
